In [3]:
!git colne https://github.com/renad-ayman/Movie-review-IMDB-.git

git: 'colne' is not a git command. See 'git --help'.

The most similar command is
	clone


In [4]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [7]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [8]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
def preprocess_text(text):
    # Remove special characters and numbers
    text = re.sub(r'\W', ' ', text)
    # Remove single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase
    text = text.lower()

    return text

# Apply preprocessing to the review column
data['review'] = data['review'].apply(preprocess_text)

# Remove stopwords using NLTK
stop_words = set(stopwords.words('english'))
data['review'] = data['review'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [ ]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
data.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,1
1,wonderful little production br br filming tech...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1


In [ ]:
X = data['review']
y = data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
data.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,1
1,wonderful little production br br filming tech...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1


In [ ]:
print(X_train.shape)
print(X_test.shape)

(40000,)
(10000,)


In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_padded = pad_sequences(X_train_seq, padding='post', maxlen=200)
X_test_padded = pad_sequences(X_test_seq, padding='post', maxlen=200)

In [ ]:
Y_train = y_train
Y_test = y_test

In [ ]:
print(X_train)

39087    kept asking many fights screaming matches swea...
30893    watch entire movie could watch entire movie st...
45278    touching love story reminiscent mood love draw...
16398    latter day fulci schlocker totally abysmal con...
13653    first firmly believe norwegian movies continua...
                               ...                        
11284    shadow magic recaptures joy amazement first mo...
44732    found movie quite enjoyable fairly entertainin...
38158    avoid one terrible movie exciting pointless mu...
860      production quite surprise absolutely love obsc...
15795    decent movie although little bit short time pa...
Name: review, Length: 40000, dtype: object


In [ ]:
print(X_test)

33553    really liked summerslam due look arena curtain...
9427     many television shows appeal quite many differ...
199      film quickly gets major chase scene ever incre...
12447    jane austen would definitely approve one br br...
39489    expectations somewhat high went see movie thou...
                               ...                        
28567    although casper van dien michael rooker genera...
25079    liked movie really sure started watching enjoy...
18707    yes non singaporean see big deal film referenc...
15200    far films go likable enough entertaining chara...
5857     saw anatomy years ago dubbed friends house rem...
Name: review, Length: 10000, dtype: object


In [ ]:
print(Y_train)

39087    0
30893    0
45278    0
16398    0
13653    0
        ..
11284    0
44732    0
38158    0
860      0
15795    0
Name: sentiment, Length: 40000, dtype: int64


In [ ]:
from tensorflow.keras.layers import Dropout

model = Sequential([
Embedding(input_dim=5000, output_dim=128, input_length=200),
LSTM(128, return_sequences=False),
Dropout(0.5),
Dense(1, activation='sigmoid')])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

max_words = 5000
max_length = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)

sequences = tokenizer.texts_to_sequences(X)

X_padded = pad_sequences(sequences, maxlen=max_length)

X_train, X_val, y_train, y_val = train_test_split(X_padded, y, test_size=0.2, random_state=42)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, callbacks=[early_stopping])

Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.9956 - loss: 0.0274 - val_accuracy: 1.0000 - val_loss: 8.0372e-07
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 1.0000 - loss: 1.8609e-06 - val_accuracy: 1.0000 - val_loss: 2.7290e-07
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 1.0000 - loss: 7.8430e-07 - val_accuracy: 1.0000 - val_loss: 1.1970e-07
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 1.0000 - loss: 3.9775e-07 - val_accuracy: 1.0000 - val_loss: 5.7893e-08
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 1.0000 - loss: 2.2253e-07 - val_accuracy: 1.0000 - val_loss: 2.8655e-08
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 1.0000 - loss: 1.2678e-07 - val_accuracy: 1.0000 - val_loss: 1.4458e-08
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 1.0000 - loss: 6.4073e-08 - val_accuracy: 1.0000 - val_loss: 7.3428e-09
Epoch 8/20
1250/1250 ━━━

In [ ]:
from sklearn.metrics import accuracy_score

# Predict on the test data
y_pred = model.predict(X_test_padded)

# Calculate the accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Test Accuracy: 100.00%
